# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [1]:
class Problem:
    @property
    def initial_state(self):
        ...

    def available_actions(self, state):
        ...

    def do_action(self, state, action):
        ...
        return new_state

    def is_goal(self, state) -> bool:
        ...

    def action_cost(self, state, action) -> float:
        ...

    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (1, (True, True))

    def available_actions(self, state):
        return ["Left", "Suck", "Right"]

    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot - 1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot + 1, len(dirty) - 1), dirty)
        raise Exception("Invalid action")

    def is_goal(self, state) -> bool:
        return not any(state[1])

    def action_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [3]:
import heapq

In [14]:
def astar(problem: Problem):
    initial_state = problem.initial_state
    visited_states = []
    q = [(problem.heuristic(initial_state), (0, [], initial_state))]
    while q:
        priority, element = heapq.heappop(q)
        cur_cost, moves, last_state = element
        if tuple(last_state) in visited_states:
            continue
        visited_states.append(tuple(last_state))
        if problem.is_goal(last_state):
            return moves
        actions = problem.available_actions(last_state)
        for action in actions:
            next_state = problem.do_action(last_state, action)
            if tuple(next_state) not in visited_states:
                action_cost = problem.action_cost(last_state, action)
                cost_after_action = cur_cost + action_cost
                cost_with_heuristic = cost_after_action + problem.heuristic(next_state)
                next_moves = moves + [action]
                que_candidate = (cost_with_heuristic, (cost_after_action, next_moves, next_state))
                heapq.heappush(q, que_candidate)

Now lets test your code in the vacuum world!

In [15]:
%timeit astar(VacuumProblem())

14.8 µs ± 791 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [16]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return len(state[1]) - sum(state[1])


%timeit astar(VacuumProblem1())

19.4 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


And another in which heuristic grossly overestimates the cost.

In [17]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return 10 * sum(state[1])


%timeit astar(VacuumProblem2())

9.85 µs ± 812 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

Although VacumProblem2 is the best it is not the expected answer, because it does not satisfy the properties of heurystic, due to the fact that it overestimates the distance to sollution. It works better than the only heurystic from VacumProblem, because it gives a lot of points to an aption that does Suck first, which in this setting is the best possible move.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [18]:
class PuzzleProblem(Problem):
    def __init__(self, init_state, goal_state):
        self.init_state = init_state
        self.goal_state = goal_state
        
    @property
    def initial_state(self):
        return self.init_state
    
    def get0index(self,state):
        for i, x in enumerate(state):
            if 0 in x:
                return (i, x.index(0))
    
    def available_actions(self, state):
        i,j = self.get0index(state);
        result =[]
        if j != 0:
            result.append('L')
        if j!=2:
            result.append('R')
        if i != 0:
            result.append('U')
        if i !=2:
            result.append('D')
        return result
        
    def do_action(self, state, action):
        i,j = self.get0index(state);
        new_state = [x[:] for x in state]
        if action == 'L':
            if j==0:
                raise Exception('Invalid action')
            new_state[i][j], new_state[i][j-1] = new_state[i][j-1],new_state[i][j]
        elif action == 'R':
            if j==2:
                raise Exception('Invalid action')
            new_state[i][j], new_state[i][j+1] = new_state[i][j+1],new_state[i][j]
        elif action == 'U':
            if i==0:
                raise Exception('Invalid action')
            new_state[i][j], new_state[i-1][j] = new_state[i-1][j], new_state[i][j]
        elif action == 'D':
            if i==2:
                raise Exception('Invalid action')
            new_state[i][j], new_state[i+1][j] = new_state[i+1][j], new_state[i][j]
        else:
            raise Exception('Invalid action') 
        return new_state
    
    def is_goal(self, state) -> bool:
        return state==self.goal_state

    def action_cost(self, state, action) -> float:
        return 1

    def manhatan_distances_of_elements(self, state) -> float:
        current_locations = [0] * 9
        for i in range(3):
            for j in range(3):
                current_locations[state[i][j]] = (i, j)
        goal_locations = [0] * 9
        for i in range(3):
            for j in range(3):
                goal_locations[self.goal_state[i][j]] = (i, j)
        return [
            abs(current_locations[i][0] - goal_locations[i][0])
            + abs(current_locations[i][1] - goal_locations[i][1])
            for i in range(1, 9)
        ]
    
    def euklidean_distances_of_elements(self, state) -> float:
        current_locations = [0] * 9
        for i in range(3):
            for j in range(3):
                current_locations[state[i][j]] = (i, j)
        goal_locations = [0] * 9
        for i in range(3):
            for j in range(3):
                goal_locations[self.goal_state[i][j]] = (i, j)
        return [
            sqrt((current_locations[i][0] - goal_locations[i][0])**2
            + (current_locations[i][1] - goal_locations[i][1])**2)
            for i in range(1, 9)
        ]

    def heuristic(self, state) -> float:
        return sum(self.manhatan_distances_of_elements(state))

**Prove that this heuristic is admissible.**

Heurystic is sum of distances of every element from its current location to desired location. Because every action that we do reduces this distance by at most 1 for 1 element our heurystic will always be smaller than real distance

In [19]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:
        m_dists = self.manhatan_distances_of_elements(state)
        eukl_dists = self.euklidean_distances_of_elements(state)
        return (sum(m_dists)+sum(eukl_dists))/2

**Prove that this heuristic is admissible.**

This heurystic is always smaller or equal to the first one

In [20]:
from math import sqrt
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        distances = self.euklidean_distances_of_elements(state) 
        return sum(distances)

**Prove that this heuristic is admissible.**

This heurystic is also always smaller or equal to the 1st one.

Run your heuristics on the given problem.

In [21]:
initial_state = [[7,2,4],[5,0,6],[8,3,1]]
goal_state = [[0,1,2],[3,4,5],[6,7,8]]
%timeit plan0 = astar(PuzzleProblem(initial_state,goal_state))
plan0 = astar(PuzzleProblem(initial_state,goal_state))
print("p0",len(plan0))

1.55 s ± 9.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
p0 26


In [22]:
%timeit plan1 = astar(PuzzleProblem1(initial_state,goal_state))
plan1 = astar(PuzzleProblem1(initial_state,goal_state))
print("p1", len(plan1))

1.75 s ± 6.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
p1 26


In [ ]:
%timeit astar(PuzzleProblem2(initial_state,goal_state))
plan2 = astar(PuzzleProblem2(initial_state,goal_state))
print("p2", len(plan2))

print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

**Which of the heuristics is the best for this task? Why is that?**

1st heurystic is the best, because it is closest to the required number of moves, because manhatan distance is greater than euklidean distance and greater than average of those 2.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.